In [11]:
import torch.nn as nn
import torch 

In [3]:
class FMEmbedding(nn.Module):
    r""" Embedding for token fields.

    Args:
        field_dims: list, the number of tokens in each token fields
        offsets: list, the dimension offset of each token field
        embed_dim: int, the dimension of output embedding vectors

    Input:
        input_x: tensor, A 3D tensor with shape:``(batch_size,field_size)``.

    Return:
        output: tensor,  A 3D tensor with shape: ``(batch_size,field_size,embed_dim)``.
    """

    def __init__(self, field_dims, offsets, embed_dim):
        super(FMEmbedding, self).__init__()
        self.embedding = nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = offsets

    def forward(self, input_x):
        input_x = input_x + input_x.new_tensor(self.offsets).unsqueeze(0)
        output = self.embedding(input_x)
        return output

In [8]:
field_dims =[4, 3]
offsets=[0, 4]
embed_dim = 4 
FMEmbedding(field_dims, offsets, embed_dim)(2)

AttributeError: 'int' object has no attribute 'new_tensor'

In [16]:
class BaseFactorizationMachine(nn.Module):
    r"""Calculate FM result over the embeddings

    Args:
        reduce_sum: bool, whether to sum the result, default is True.

    Input:
        input_x: tensor, A 3D tensor with shape:``(batch_size,field_size,embed_dim)``.

    Output
        output: tensor, A 3D tensor with shape: ``(batch_size,1)`` or ``(batch_size, embed_dim)``.
    """

    def __init__(self, reduce_sum=True):
        super(BaseFactorizationMachine, self).__init__()
        self.reduce_sum = reduce_sum

    def forward(self, input_x):
        square_of_sum = torch.sum(input_x, dim=1) ** 2
        print(square_of_sum)
        sum_of_square = torch.sum(input_x ** 2, dim=1)
        print(sum_of_square)
        output = square_of_sum - sum_of_square
        if self.reduce_sum:
            output = torch.sum(output, dim=1, keepdim=True)
        output = 0.5 * output
        return output

In [42]:
testor = torch.rand(1, 2, 4)
print(testor)
baseFM = BaseFactorizationMachine()
baseFM(testor)

tensor([[[0.8637, 0.8683, 0.5969, 0.1408],
         [0.1930, 0.7096, 0.0815, 0.7405]]])
tensor([[1.1167, 2.4898, 0.4602, 0.7767]])
tensor([[0.7832, 1.2575, 0.3630, 0.5682]])


tensor([[0.9358]])

In [45]:
testor**2

tensor([[[0.7459, 0.7539, 0.3563, 0.0198],
         [0.0373, 0.5036, 0.0066, 0.5484]]])

In [43]:
torch.sum(testor, dim=1)

tensor([[1.0567, 1.5779, 0.6784, 0.8813]])

In [37]:
torch.sum(testor, dim=1)  * torch.sum(testor, dim=1)

tensor([[3.3762, 0.7137, 0.8567, 0.7068],
        [0.9635, 0.9119, 0.2637, 0.2471],
        [0.4506, 2.9331, 0.2377, 0.2812],
        [0.9868, 1.8732, 1.3557, 1.4759]])

In [36]:
a = [1.8374, 0.8448, 0.9256, 0.8407]
b = [1.8374,0.9816,0.6713,0.994]

for i, j in zip(a, b):
    print(i * j)

3.3760387599999997
0.82925568
0.62135528
0.8356558


In [31]:
sum(testor[0][1]* testor[0][0])

tensor(1.2784)